# ETL

CSV to Parquet
- Normalize schema
- Cleanup string fields (`payment_type`, `store_and_fwd_flag`) 
- Write to parquet

In [2]:
import s3fs
import dask.dataframe as dd
import numpy as np
import pandas as pd
import os

base_path = 's3://nyc-tlc/trip data/yellow_tripdata'
if 'TAXI_S3' not in os.environ:
    raise ValueError('Set TAXI_S3 environment variable to an S3 location that you have read/write access to')
taxi_path = os.environ['TAXI_S3']

In [1]:
from dask.distributed import Client
from dask_saturn import SaturnCluster
import time

cluster = SaturnCluster(n_workers=10, scheduler_size='xlarge', worker_size='8xlarge', nthreads=32)
client = Client(cluster)

cluster

[2020-08-04 12:21:45] INFO - dask-saturn | Starting cluster. Status: stopped
[2020-08-04 12:21:53] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:22:05] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:22:43] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:23:28] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:24:24] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:24:56] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:25:52] INFO - dask-saturn | Starting cluster. Status: starting
[2020-08-04 12:26:49] INFO - dask-saturn | Cluster is ready


In [3]:
fs = s3fs.S3FileSystem(anon=True)

In [4]:
cols_types = [
    ('vendor_id', 'string'),
    ('pickup_datetime', 'object'),
    ('dropoff_datetime', 'object'),
    ('passenger_count', 'float64'),
    ('trip_distance', 'float64'),
    ('rate_code_id', 'float64'),
    ('store_and_fwd_flag', 'object'),
    ('pickup_taxizone_id',  'float64'),
    ('dropoff_taxizone_id', 'float64'),
    ('pickup_latitude', 'float64'),
    ('pickup_longitude', 'float64'),
    ('dropoff_latitude', 'float64'),
    ('dropoff_longitude', 'float64'),
    ('payment_type', 'object'),
    ('fare_amount', 'float64'),
    ('extra', 'float64'),
    ('mta_tax', 'float64'),
    ('tip_amount', 'float64'),
    ('tolls_amount', 'float64'),
    ('improvement_surcharge', 'float64'),
    ('total_amount', 'float64'),
    ('congestion_surcharge', 'float64'),
]
dtypes = dict(cols_types)
all_cols = [x[0] for x in cols_types]

file_specs = {
    'pre2015': {
        'date_range': ('2009-01', '2015-01'),
        'cols': [
            'vendor_id',
            'pickup_datetime',
            'dropoff_datetime',
            'passenger_count',
            'trip_distance',
            'pickup_longitude',
            'pickup_latitude',
            'rate_code_id',
            'store_and_fwd_flag',
            'dropoff_longitude',
            'dropoff_latitude',
            'payment_type',
            'fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'total_amount',
        ],
    },
    '2015_mid2016': {
        'date_range': ('2015-01', '2016-07'),
        'cols': [
            'vendor_id',
            'pickup_datetime',
            'dropoff_datetime',
            'passenger_count',
            'trip_distance',
            'pickup_longitude',
            'pickup_latitude',
            'rate_code_id',
            'store_and_fwd_flag',
            'dropoff_longitude',
            'dropoff_latitude',
            'payment_type',
            'fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'improvement_surcharge',
            'total_amount',
        ]
    },
    'late2016': {
        'date_range': ('2016-07', '2017-01'),
        'cols': [
            'vendor_id',
            'pickup_datetime',
            'dropoff_datetime',
            'passenger_count',
            'trip_distance',
            'rate_code_id',
            'store_and_fwd_flag',
            'pickup_taxizone_id',
            'dropoff_taxizone_id',
            'payment_type',
            'fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'improvement_surcharge',
            'total_amount',
            'junk1',  # extra trailing commas
            'junk2',
        ]
    },
    '2017_2018': {
        'date_range': ('2017-01', '2019-01'),
        'cols': [
            'vendor_id',
            'pickup_datetime',
            'dropoff_datetime',
            'passenger_count',
            'trip_distance',
            'rate_code_id',
            'store_and_fwd_flag',
            'pickup_taxizone_id',
            'dropoff_taxizone_id',
            'payment_type',
            'fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'improvement_surcharge',
            'total_amount',
        ]
    },
    '2019': {
        'date_range': ('2019-01', '2020-01'),
        'cols': [
            'vendor_id',
            'pickup_datetime',
            'dropoff_datetime',
            'passenger_count',
            'trip_distance',
            'rate_code_id',
            'store_and_fwd_flag',
            'pickup_taxizone_id',
            'dropoff_taxizone_id',
            'payment_type',
            'fare_amount',
            'extra',
            'mta_tax',
            'tip_amount',
            'tolls_amount',
            'improvement_surcharge',
            'total_amount',
            'congestion_surcharge',
        ]
    }
}

def get_files(date_range):
    return [f'{base_path}_{x}.csv' 
            for x in pd.date_range(date_range[0], date_range[1], freq='M').strftime("%Y-%m")]

In [18]:
def load_csv(date_range, cols):
    df = dd.read_csv(
        get_files(date_range),
        header=0,
        na_values=["NA"],
        parse_dates=[1, 2],  # dates are in same position for each file
        infer_datetime_format=True,
        dtype=dtypes,
        names=cols,
        warn_bad_lines=False,
        error_bad_lines=False,
        storage_options={'anon': True},
    )
    
    missing_cols = set(all_cols) - set(cols)
    for c in missing_cols:
        df[c] = np.nan
        df[c] = df[c].astype(dtypes[c])
            
    df = df[all_cols]
    return df

In [19]:
dfs = []
for which, meta in file_specs.items():
    dfs.append(load_csv(meta['date_range'], meta['cols']))

In [22]:
all_taxi = dd.concat(dfs)
all_taxi

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_taxizone_id,dropoff_taxizone_id,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=3965,,,,,,,,,,,,,,,,,,,,,,
,string,datetime64[ns],datetime64[ns],float64,float64,float64,object,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
payment_map = {
    '1': 1,
    'CRD': 1,
    'Credit': 1,
    'Cre': 1,
    'CRE': 1,
    'CREDIT': 1,
    '2': 2,
    'CSH': 2,
    'CASH': 2,
    'Cash': 2,
    'CAS': 2,
    'Cas': 2,
    '3': 3,
    'NOC': 3,
    'No Charge': 3,
    'No': 3,
    '4': 4,
    'DIS': 4,
    'Dispute': 4,
    'Dis': 4,
    '5': 5,
    'UNK': 5,
    'NA': 5,
}

all_taxi['store_and_fwd_flag'] = all_taxi.store_and_fwd_flag.isin({'1', 'Y'}).astype(int)
all_taxi['payment_type'] = all_taxi.payment_type.map(payment_map).fillna(5).astype(int)

In [24]:
%%time
(all_taxi
 .repartition(npartitions=400)
 .to_parquet(f'{taxi_path}/data/taxi_parquet', 
             engine='pyarrow', compression='snappy'))

CPU times: user 12.9 s, sys: 253 ms, total: 13.2 s
Wall time: 9min 9s


# Check output

In [4]:
taxi = dd.read_parquet(f'{taxi_path}/data/taxi_parquet', engine='pyarrow')

In [5]:
taxi.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_taxizone_id,dropoff_taxizone_id,pickup_latitude,...,dropoff_longitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1.0,2.63,NaN,0,NaN,NaN,40.721567,...,-73.993803,2,8.9,0.5,NaN,0.00,0.0,NaN,9.40,NaN
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3.0,4.55,NaN,0,NaN,NaN,40.736290,...,-73.955850,1,12.1,0.5,NaN,2.00,0.0,NaN,14.60,NaN
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5.0,10.35,NaN,0,NaN,NaN,40.739748,...,-73.869983,1,23.7,0.0,NaN,4.74,0.0,NaN,28.44,NaN
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1.0,5.00,NaN,0,NaN,NaN,40.790955,...,-73.996558,1,14.9,0.5,NaN,3.05,0.0,NaN,18.45,NaN
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1.0,0.40,NaN,0,NaN,NaN,40.719382,...,-74.008378,2,3.7,0.0,NaN,0.00,0.0,NaN,3.70,NaN


In [7]:
taxi.dtypes

vendor_id                        string
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
passenger_count                 float64
trip_distance                   float64
rate_code_id                    float64
store_and_fwd_flag                int64
pickup_taxizone_id              float64
dropoff_taxizone_id             float64
pickup_latitude                 float64
pickup_longitude                float64
dropoff_latitude                float64
dropoff_longitude               float64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [8]:
%%time
len(taxi)

CPU times: user 117 ms, sys: 12.1 ms, total: 129 ms
Wall time: 7.95 s


1611604226

In [9]:
%%time
taxi.payment_type.value_counts().compute()

CPU times: user 170 ms, sys: 4.05 ms, total: 174 ms
Wall time: 8.21 s


1    843887084
2    760449294
3      4340090
5      1557129
4      1370629
Name: payment_type, dtype: int64